In [1]:
import openai
from qdrant_client import QdrantClient

In [2]:
# Define the embedding model to use
embedding_model = "text-embedding-3-small"
# Define the collection name in qdrant database
collection_name = "Amazon-items-collection-00"
# Define the OpenaI language model to use
model = "gpt-5-nano"

In [3]:
# Embedding function; same from previous notebook
def get_embedding(text, model=embedding_model):
    response = openai.embeddings.create(
        input=text,
        model=model,
    )
    return response.data[0].embedding

In [4]:
# Create the Qdrant client (ensure first to have Qdrant server running locally via docker compose;
# Then define the function for data retrieval from Qdrant based on a query text
# top K similar items will be retrieved
qdrant_client = QdrantClient(url="http://localhost:6333")
def retrieve_data(query, qdrant_client, k=5):

    query_embedding = get_embedding(query)

    results = qdrant_client.query_points(
        collection_name=collection_name,
        query=query_embedding,
        limit=k,
    )

    retrieved_context_ids = []
    retrieved_context = []
    similarity_scores = []
    retrieved_context_ratings = []

    for result in results.points:
        retrieved_context_ids.append(result.payload["parent_asin"])
        retrieved_context.append(result.payload["description"])
        retrieved_context_ratings.append(result.payload["average_rating"])
        similarity_scores.append(result.score)

    return {
        "retrieved_context_ids": retrieved_context_ids,
        "retrieved_context": retrieved_context,
        "retrieved_context_ratings": retrieved_context_ratings,
        "similarity_scores": similarity_scores,
    }

In [5]:
# Finally test the retrieval function
retrieved_context = retrieve_data(
    "What kind of earphones can I get?", qdrant_client, k=10
)
# And print the context retrieved
retrieved_context

{'retrieved_context_ids': ['B0CBMPG524',
  'B0B9FTVL58',
  'B0C142QS8X',
  'B0C6K1GQCF',
  'B0BNHVLF7G',
  'B0B14HTZ59',
  'B09WCFC5D9',
  'B0B67ZFRPC',
  'B0B7495RL6',
  'B0BS15TRJ3'],
 'retrieved_context': ['Open Ear Headphones, Bluetooth 5.3 Earbuds with 60H Playtime IPX7 Waterproof Wireless Earbuds Immersive Premium Sound True Wireless Open Ear Earbuds with Earhooks for Running, Walking and Workouts 【Open-ear Design Headphones】Feature with a new generation of true open-ear wireless earbuds design, the headphones can rest gently and firmly fit your ears without entering your ear canal, which will reduce stress and hearing loss after extended wear. There is no pinching of the auricle, no blockage of the ear canal, and no pain or damage to hearing. 【Powerful Stereo Sound】Equipped with 16.2 millimeters vibrating diaphragm speaker driver, bluetooth headphones providing pure balanced audio and clarity output for all music genres with soft audio and immerse yourself in the wonderful world

In [6]:
# Now format the context before using it inside the prompt
def process_context(context):

    formatted_context = ""

    for id, chunk, rating in zip(
        context["retrieved_context_ids"],
        context["retrieved_context"],
        context["retrieved_context_ratings"],
    ):
        formatted_context += f"- ID: {id}, rating: {rating}, description: {chunk}\n"

    return formatted_context

preprocessed_context = process_context(retrieved_context)
print(preprocessed_context)

- ID: B0CBMPG524, rating: 4.4, description: Open Ear Headphones, Bluetooth 5.3 Earbuds with 60H Playtime IPX7 Waterproof Wireless Earbuds Immersive Premium Sound True Wireless Open Ear Earbuds with Earhooks for Running, Walking and Workouts 【Open-ear Design Headphones】Feature with a new generation of true open-ear wireless earbuds design, the headphones can rest gently and firmly fit your ears without entering your ear canal, which will reduce stress and hearing loss after extended wear. There is no pinching of the auricle, no blockage of the ear canal, and no pain or damage to hearing. 【Powerful Stereo Sound】Equipped with 16.2 millimeters vibrating diaphragm speaker driver, bluetooth headphones providing pure balanced audio and clarity output for all music genres with soft audio and immerse yourself in the wonderful world of music. 【Hear Your Surroundings】Open earbuds that rest on your ears without covering them, you can hear your music and your surroundings at the same time. Whether 

In [7]:
# Create the prompt with the enriched context
def build_prompt(preprocessed_context, question):

    prompt = f"""
You are a shopping assistant that can answer questions about the products in stock.

You will be given a question and a list of context.

Instructions:
- You need to answer the question based on the provided context only.
- Never use word context and refer to it as the available products.

Context:
{preprocessed_context}

Question:
{question}
"""

    return prompt

In [8]:
# Build the prompt starting from the original user query
prompt = build_prompt(preprocessed_context, "What kind of earphones can I get?")
print(prompt)


You are a shopping assistant that can answer questions about the products in stock.

You will be given a question and a list of context.

Instructions:
- You need to answer the question based on the provided context only.
- Never use word context and refer to it as the available products.

Context:
- ID: B0CBMPG524, rating: 4.4, description: Open Ear Headphones, Bluetooth 5.3 Earbuds with 60H Playtime IPX7 Waterproof Wireless Earbuds Immersive Premium Sound True Wireless Open Ear Earbuds with Earhooks for Running, Walking and Workouts 【Open-ear Design Headphones】Feature with a new generation of true open-ear wireless earbuds design, the headphones can rest gently and firmly fit your ears without entering your ear canal, which will reduce stress and hearing loss after extended wear. There is no pinching of the auricle, no blockage of the ear canal, and no pain or damage to hearing. 【Powerful Stereo Sound】Equipped with 16.2 millimeters vibrating diaphragm speaker driver, bluetooth headp

In [9]:
# Generate the answer with the enriched prompt
def generate_answer(prompt):

    response = openai.chat.completions.create(
        model=model,
        messages=[{"role": "system", "content": prompt}],
        reasoning_effort="minimal",
    )

    return response.choices[0].message.content

print(generate_answer(prompt))

You can get various wireless/open-ear and in-ear earphones, including:

- Open Ear / open-ear design wireless earbuds with earhooks (comfortable, safe for hearing, hear surroundings) like the Open Ear Headphones with IPX7 and long playback up to 60 hours with case.
- Wireless Bluetooth 5.3 in-ear earbuds with IPX7 waterproofing and long playback (37 or 30 hours depending on model).
- Bone conduction open-ear headphones with bone conduction tech (X15) for sport use.
- Kids’ over-ear wired headphones with volume limit, plus 3.5mm compatibility.
- Game/low-latency true wireless earbuds with 5.2/5.3 Bluetooth and low latency modes.
If you want a specific style or features (open-ear, bone conduction, kids’ headphones, waterproof, or gaming mode), tell me what you prefer and I’ll point to suitable options.


In [10]:
# Create a naive RAG pipeline that assembles all the previous steps
def rag_pipeline(question, top_k=5):

    qdrant_client = QdrantClient(url="http://localhost:6333")

    retrieved_context = retrieve_data(question, qdrant_client, top_k)
    preprocessed_context = process_context(retrieved_context)
    prompt = build_prompt(preprocessed_context, question)
    answer = generate_answer(prompt)

    return answer

# Finally test the retrieval function
print(rag_pipeline("What kind of earphones can I get with ratings above 4.5?"))

Based on the available products, the earphones with a rating above 4.5 are:

- ID B0C142QS8X: TUNEAKE Kids Headphones, rating 4.5 (exactly 4.5, not above)
- None listed with a rating strictly above 4.5 (i.e., > 4.5)

If you’re okay with 4.5 or higher, the TUNEAKE Kids Headphones meet that threshold at 4.5.
